# Imports and packages

In [1]:
#!pip install spacy -q
#!python -m spacy download en_core_web_sm -q
#!pip install vaderSentiment
#!pip install spacy
## helpful packages

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
#! python3 -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

import nltk
#nltk.download("stopwords")
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/madhvimalhotra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/madhvimalhotra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Text analysis of Department of Justice (DOJ) press releases (50 points total)

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases
    
- See here for a codebook: https://docs.google.com/spreadsheets/d/1UopmSvFGrwJvz_c3Plh32Yxkqwff64oS_CcpfATOV8k/edit?usp=sharing

In [2]:
## first, unzip the combined.json.zip file
## then, run this code to load the unzipped json file and convert to a dataframe
## and convert some of the attributes from lists to values
## make sure to change the pathname if you need to
doj = pd.read_json("combined.json", lines = True)
#doj
## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()


id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (16 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string

#doj[doj["id"]=="17-1204"]

pharma=doj[doj["id"]=="17-1204"]["contents"].iloc[0]

pharma

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (so can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for .isalpha(): https://www.w3schools.com/python/ref_string_isalpha.asp
- `process_step1` function here has an example of tokenizing and filtering to words where .isalpha() is true: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb 
- Part of speech tagging section of this code: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb



In [4]:
## your code here to restrict to alpha
#pharma.isalpha()

pharma_1= [word for word in pharma.split(" ") if word.isalpha() == True]

pharma_1 = ' '.join(str(x) for x in pharma_1)

#pharma_1
pharma_1_tokens = pos_tag(pharma_1.split(" "))

pharma_1_tokens

[('The', 'DT'),
 ('founder', 'NN'),
 ('and', 'CC'),
 ('majority', 'NN'),
 ('owner', 'NN'),
 ('of', 'IN'),
 ('Insys', 'NNP'),
 ('Therapeutics', 'NNP'),
 ('was', 'VBD'),
 ('arrested', 'VBN'),
 ('today', 'NN'),
 ('and', 'CC'),
 ('charged', 'VBN'),
 ('with', 'IN'),
 ('leading', 'VBG'),
 ('a', 'DT'),
 ('nationwide', 'JJ'),
 ('conspiracy', 'NN'),
 ('to', 'TO'),
 ('profit', 'VB'),
 ('by', 'IN'),
 ('using', 'VBG'),
 ('bribes', 'NNS'),
 ('and', 'CC'),
 ('fraud', 'NN'),
 ('to', 'TO'),
 ('cause', 'VB'),
 ('the', 'DT'),
 ('illegal', 'JJ'),
 ('distribution', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('Fentanyl', 'NNP'),
 ('spray', 'NN'),
 ('intended', 'VBD'),
 ('for', 'IN'),
 ('cancer', 'NN'),
 ('patients', 'NNS'),
 ('experiencing', 'VBG'),
 ('breakthrough', 'IN'),
 ('than', 'IN'),
 ('Americans', 'NNPS'),
 ('died', 'VBD'),
 ('of', 'IN'),
 ('synthetic', 'JJ'),
 ('opioid', 'NN'),
 ('overdoses', 'NNS'),
 ('last', 'JJ'),
 ('and', 'CC'),
 ('millions', 'NNS'),
 ('are', 'VBP'),
 ('addicted', 'VBN'),
 ('to', 'T

In [5]:
## your code here for part of speech tagging

pharma_2_tokens = pd.DataFrame(pharma_1_tokens,columns=["word","POS"])

#pharma_2_tokens

pharma_2_tokens[pharma_2_tokens["POS"].isin(["JJ","JJR",
                                             "JJS"])].groupby("word")["POS"].count().reset_index().sort_values(
    by="POS",ascending=False).head(5)



,word,POS
14,former,8
33,nationwide,4
36,other,3
3,addictive,3
35,opioid,3


## 1.2 named entity recognition (3 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`. Here's some background on what RICO means: https://www.justia.com/criminal/docs/rico/ 

**Resources**:
- For parts A and B: named entity recognition part of this code: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb

In [6]:
## your code here 

spacy_pharma = nlp(pharma)
print(type(spacy_pharma))

#spacy_pharma.ents


for one_tok in spacy_pharma.ents:
    if one_tok.label_== "LAW":
        print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)
    

<class 'spacy.tokens.doc.Doc'>
Entity: RICO; NER tag: LAW
Entity: the Controlled Substances Act; NER tag: LAW
Entity: RICO; NER tag: LAW


C. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years. Print these named entities.

**Hint:**  
You may want to use the `re` module for the second part.

In [7]:
## your code here
output=[]
for one_tok in spacy_pharma.ents:
    if one_tok.label_== "DATE" and "year" in str(one_tok.text):
        print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)
        output.append(one_tok.text)

output

Entity: last year; NER tag: DATE
Entity: three years; NER tag: DATE
Entity: three years; NER tag: DATE


['last year', 'three years', 'three years']

D. Parse the pharma string at the sentence level. Note that this involves more than just splitting on each `.`; for full credit, add at least one additional delimiter that marks the end of the sentence.

Then, using those sentences, pull and print the original sentences from the press releases where those year lengths are mentioned. Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint:**  
You may want to use re.search or re.findall 

**Resources**:

- re.search and re.findall examples here for filtering to ones containing year (multiple approaches; some need not involve `re`): 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/07_regex_solutions.ipynb


In [8]:
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#fp = open("test.txt")
#data = fp.read()
output1 = ('n-----n'.join(tokenizer.tokenize(pharma)))

output2=output1.split('n-----n')

for sentence in output2:
    print(sentence,"\n")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/madhvimalhotra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain. 

"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. 

And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. 

"This Justice Department will not tolerate this. 

We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic. 

And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspira

## 1.3 sentiment analysis  (10 points)

- Sentiment analysis section of this script: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb


A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [9]:
## your code here for subsetting
doj_subset=doj[doj["topics_clean"].isin(["Civil Rights","Hate Crimes","Project Safe Childhood"])]
doj_subset.shape

(717, 6)

In [10]:
doj_subset

,id,title,contents,date,topics_clean,components_clean
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2015-12-11T00:00:00-05:00,Project Safe Childh

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint:** you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [11]:
# your code here defining the function
doj_subset.columns


Index(['id', 'title', 'contents', 'date', 'topics_clean', 'components_clean'], dtype='object')

In [12]:
# your code here executing the function

#import spacy
sent_obj = SentimentIntensityAnalyzer()
nlp = spacy.load('en_core_web_sm')
def sentiment_score(one_string):
    text_data = one_string
    document = nlp(text_data)
    text_no_namedentities = []
    ents = [e.text for e in document.ents]
    for item in document:
        if item.text in ents:
            pass 
        else:
            text_no_namedentities.append(item.text)
    removed_ner = " ".join(text_no_namedentities)
    ## score one listing
    sentiment_example = sent_obj.polarity_scores(removed_ner)
    return sentiment_example

doj_subset["sentiment_score"]= doj_subset["contents"].apply(lambda x: sentiment_score(x))
doj_subset


,id,title,contents,date,topics_clean,components_clean,sentiment_score
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle","{'neg': 0.159, 'neu': 0.777, 'pos': 0.064, 'compound': -0.9893}"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score (so most negative to least negative) and print the `id`, `contents`, and `neg` columns of the two most negative press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [13]:
# you rcode here

doj_subset_wscore = doj_subset.copy(deep=True)

doj_subset_wscore["neg"] = doj_subset_wscore["sentiment_score"].apply(lambda x: x["neg"])

doj_subset_wscore.sort_values(["neg"],ascending=False,inplace=True)

doj_subset_wscore[["id","contents","neg"]].head()

,id,contents,neg
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.269
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",0.259
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dang

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [14]:
## agg and find the mean compound score by topic

doj_subset_wscore["compound_score"] = doj_subset_wscore["sentiment_score"].apply(lambda x: x["compound"])


topics_clean = doj_subset_wscore.groupby("topics_clean").agg({"compound_score":"mean"})

topics_clean


,compound_score
topics_clean,
Civil Rights,0.131907
Hate Crimes,-0.893759
Project Safe Childhood,-0.227048


# YOUR INTERPRETATION HERE
The compound score is the sum of positive, negative & neutral scores which is then normalized between -1(most extreme negative) and +1 (most extreme positive).

There is a variation in score because there is more negative sentiment attached to Hate Crimes and Project Safe childhood as both are negative and a positive sentiment towards Civil rights.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in the code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string (so if press release is something like "The CEO was indicted" it might return "ceo indict" 
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`. Note: there will be a deduction if your code uses a non-list comprehension for loop that uses append.
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/
- Here's code with topic modeling steps: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb

In [15]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]
list_stopwords = stopwords.words("english")
list_stopwords_new = list_stopwords + custom_doj_stopwords

In [16]:
## your code defining a text processing function
porter = PorterStemmer()
#content1 = doj_subset_wscore["contents"].iloc[0]

def preprocess(content1):
    content1 = content1.lower()
    nonstop_listing = ' '.join([porter.stem(word) 
                for word in wordpunct_tokenize(content1) 
                if word not in list_stopwords_new
                and word.isalpha() == True
                and len(word)>4])
    return nonstop_listing
## your code executing the function
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(lambda x: preprocess(x))

doj_subset_wscore[["id","contents","processed_text"]]

,id,contents,processed_text
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",announc morn albuquerqu initi appear feder court crimin complaint charg crime offens charg relat semit threat jewish woman oper jewish delicatessen bakeri albuquerqu arrest march base crimin complaint alleg interf victim feder protect threaten interf busi religion accord crimin complaint allegedli post threaten semit note vicin victim busi crimin complaint mere establish probabl caus presum innoc unless proven guilti convict offens charg crimin complaint face maximum statutori penalti prison matter investig albuquerqu prosecut baker mexico aejean
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison cruel unusu punish sturdiv charg fail interven marsher punch beat indict alleg action involv danger weapon res

In [17]:
## your code showing the examples
doj_subset_wscore[doj_subset_wscore["id"].isin(["16-718","16-217"])][["id","contents","processed_text"]]

,id,contents,processed_text
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison cruel unusu punish sturdiv charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three convict marsher sturdiv maximum sentenc year prison excess forc charg three offic face year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi mulhaus crimin section marsher indict
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectiv

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset two.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.

**Resources**:

- Here contains an example of applying the create_dtm function: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb


In [18]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [19]:
## your code 
dtm_nopre = create_dtm(list_of_strings= doj_subset_wscore.processed_text,
                metadata = 
                doj_subset_wscore[['id', 'compound_score', 'topics_clean']])

dtm_nopre.head()

,index,id,compound_score,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zachari,zamora,zealand,zealou,zeeman,zionism,zobel,zone,zunggeemog,zwengel
0,329,14-248,-0.9950,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11593,16-718,-0.9968,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,572,13-312,-0.9984,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11876,15-1348,-0.9967,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,501,11-626,-0.9985,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#dtm_nopre.sort_values("compound",ascending=False,inplace=True)

#df=dtm_nopre.quantile(q=0.05, axis=0, numeric_only=True,interpolation='linear').reset_index()

#df

def get_topwords(df2,col,quant,skip_cols,operator):
    quant_value = df2[col].quantile(quant)
    if operator == "GT":
        df = df2[df2[col]>quant_value]
    else:
        df = df2[df2[col]<quant_value]
    top_terms = df[[col for col in df.columns
                   if col not in skip_cols]].sum(axis=0)
    return top_terms.sort_values(ascending = False)[0:10]
    
top_terms = get_topwords(dtm_nopre,"compound_score",0.95,["index","id","topics_clean","compound_score"],"GT")    
print(top_terms)

for crime in ["Project Safe Childhood","Civil Rights","Hate Crimes"]:
    top_terms1 = get_topwords(dtm_nopre[dtm_nopre["topics_clean"]==crime],"compound_score",0.95,["index","id","topics_clean","compound_score"],"GT")    
    print("top_terms1 ", crime, " are:\n",top_terms1)
    

agreement     168
disabl        132
commun        115
enforc        114
state         107
settlement    104
ensur         101
hous           91
student        85
gener          82
dtype: int64
top_terms1  Project Safe Childhood  are:
 child          71
exploit        46
pornographi    33
children       31
sexual         30
gener          28
prosecut       26
state          25
childhood      25
project        24
dtype: int64
top_terms1  Civil Rights  are:
 agreement     105
disabl         64
settlement     62
enforc         58
ensur          56
commun         53
student        52
state          42
access         41
servic         40
dtype: int64
top_terms1  Hate Crimes  are:
 american    34
crime       31
church      29
burn        24
prosecut    22
cross       22
african     22
protect     21
investig    21
sentenc     20
dtype: int64


In [21]:
lowest_terms = get_topwords(dtm_nopre,"compound_score",0.05,["index","id","topics_clean","compound_score"],"LT")    
print(lowest_terms)

assault     181
crime       166
victim      154
offic       117
conspir     115
defend      107
american    106
sentenc     103
charg       100
african      95
dtype: int64


In [22]:
for crime in ["Project Safe Childhood","Civil Rights","Hate Crimes"]:
    lowest_terms1 = get_topwords(dtm_nopre[dtm_nopre["topics_clean"]==crime],"compound_score",0.05,["index","id","topics_clean","compound_score"],"LT")
    print("lowest_terms1 for", crime, "are:\n",lowest_terms1)
    

lowest_terms1 for Project Safe Childhood are:
 child       75
sexual      55
exploit     45
sentenc     39
children    38
abus        36
guilti      31
crimin      29
victim      28
websit      28
dtype: int64
lowest_terms1 for Civil Rights are:
 offic      157
sterl       82
feder       54
assault     52
trent       50
charg       50
victim      48
violat      45
evid        40
sentenc     37
dtype: int64
lowest_terms1 for Hate Crimes are:
 assault        83
conspir        65
american       64
anderson       57
african        56
crime          56
mississippi    54
victim         52
jackson        51
charg          48
dtype: int64


## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled topics (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [23]:
doj_subset_wscore.topics_clean.value_counts()

Civil Rights              305
Hate Crimes               246
Project Safe Childhood    166
Name: topics_clean, dtype: int64

In [24]:
## your code

## Step 1: re-tokenize and store in list

text_raw_tokens = [wordpunct_tokenize(one_text) 
                for one_text in 
                doj_subset_wscore.processed_text]

## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)

## Step 3: filter out very rare and very common words

lower_bound = round(doj_subset_wscore.shape[0]*0.05)
upper_bound = round(doj_subset_wscore.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)

# Step 4: apply dictionary to TOKENIZED texts

corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]


In [25]:
#text_raw_dict.items()

In [26]:
## Step 5: we're finally ready to estimate the model!

ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                num_topics = 3, id2word=text_raw_dict, 
                                passes=6, alpha = 'auto',
                                per_word_topics = True, random_state = 10)

In [27]:
## Post-model 1: explore corpus-wide summary of topics

topics = ldamod.print_topics(num_words = 15)
for topic in topics:
    print(topic)

(0, '0.020*"child" + 0.019*"victim" + 0.017*"prosecut" + 0.016*"sentenc" + 0.014*"charg" + 0.014*"guilti" + 0.013*"feder" + 0.013*"exploit" + 0.012*"sexual" + 0.012*"crime" + 0.011*"defend" + 0.011*"investig" + 0.011*"crimin" + 0.010*"prison" + 0.010*"count"')
(1, '0.024*"offic" + 0.015*"violat" + 0.014*"commun" + 0.013*"feder" + 0.012*"enforc" + 0.012*"state" + 0.011*"counti" + 0.011*"gener" + 0.011*"polic" + 0.010*"agreement" + 0.010*"today" + 0.009*"forc" + 0.009*"deputi" + 0.009*"court" + 0.009*"peopl"')
(2, '0.027*"hous" + 0.026*"discrimin" + 0.016*"disabl" + 0.013*"alleg" + 0.013*"enforc" + 0.012*"complaint" + 0.012*"settlement" + 0.011*"court" + 0.010*"file" + 0.010*"inform" + 0.010*"practic" + 0.010*"employ" + 0.009*"requir" + 0.009*"state" + 0.009*"agreement"')


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). 
**Hint**:    
pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

**Resources**:

- End of this code (`Additional summaries of topics and documents`) contains example of how to use `get_document_topics` and other steps to add topic probabilities back to data: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb
- If you're getting errors, use shape, len, and other commands to check the dimensionality of things at different steps since documents may be dropped if they contain no words post-processing 

In [28]:
## your code here to get doc-level topic probabilities 
## Post-model 2: explore topics associated with each document
### for each item in our original dictionary, get list of topic probabilities
l=[ldamod.get_document_topics(item) for item in corpus_fromdict]

### print result
#text_raw_tokens[0:5]

l[0:5]
len(l)

[[(0, 0.8786911), (2, 0.119364314)],
 [(0, 0.37605053), (1, 0.623106)],
 [(0, 0.83084744), (1, 0.16834147)],
 [(0, 0.7024568), (1, 0.29656315)],
 [(0, 0.9983181)]]

717

In [29]:
l[0]
l[1]

[(0, 0.8786911), (2, 0.119364314)]

[(0, 0.37605053), (1, 0.623106)]

In [30]:
pd.DataFrame(l).head(100)

,0,1,2
0,"(0, 0.8786911)","(2, 0.119364314)",None
1,"(0, 0.37605053)","(1, 0.623106)",None
2,"(0, 0.83084744)","(1, 0.16834147)",None
3,"(0, 0.7024568)","(1, 0.29656315)",None
4,"(0, 0.9983181)",None,None
...,...,...,...
95,"(0, 0.8276266)","(1, 0.1442958)","(2, 0.028077578)"
96,"(0, 0.8215083)","(1, 0.17674178)",None
97,"(0, 0.6920156)","(1, 0.3075932)",None
98,"(0, 0.9958521)",None,None


In [31]:
major_topic = []
for abc in l:
  #  print(abc)
    prob = 0
    for a in abc:
#        print(a)
        if a[1]>prob:
            topic = a[0]
    major_topic.append(topic)
    
#    if abc[0][1]>abc[1][1]:
#        topic = abc[0][0]
#    else:
#        topic = abc[1][0]
#    major_topic.append(topic)


In [32]:
len(major_topic)

717

In [33]:
doj_subset_wscore["topic"] = major_topic

In [34]:
doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,sentiment_score,neg,compound_score,processed_text,topic
329,14-248,Albuquerque Man Charged with Federal Hate Crime Related to Anti-Semitic Threats Against Businesswoman,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",2014-03-10T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,"{'neg': 0.269, 'neu': 0.678, 'pos': 0.053, 'compound': -0.995}",0.269,-0.9950,announc morn albuquerqu initi appear feder court crimin complaint charg crime offens charg relat semit threat jewish woman oper jewish delicatessen bakeri albuquerqu arrest march base crimin complaint alleg interf victim feder protect threaten interf busi religion accord crimin complaint allegedli post threaten semit note vicin victim busi crimin complaint mere establish probabl caus presum innoc unless proven guilti convict offens charg crimin complaint face maximum statutori penalti prison matter investig albuquerqu prosecut baker mexico aejean,2
11593,16-718,Three Mississippi Correctional Officers Indicted for Inmate Assault and Cover-Up,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s

In [35]:
doj_subset_wscore["topics_clean"].value_counts().reset_index()

,index,topics_clean
0,Civil Rights,305
1,Hate Crimes,246
2,Project Safe Childhood,166


In [36]:
pd.pivot_table(doj_subset_wscore,index="topics_clean", 
               values="topic", aggfunc='count', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)

,topic
topics_clean,
Civil Rights,305
Hate Crimes,246
Project Safe Childhood,166


In [37]:
df2=doj_subset_wscore.groupby(["topics_clean","topic"]).agg("count")["title"].reset_index()

df2['prob'] = df2['title'] / df2.groupby('topics_clean')['title'].transform('sum')


In [38]:
df2

,topics_clean,topic,title,prob
0,Civil Rights,0,1,0.003279
1,Civil Rights,1,107,0.350820
2,Civil Rights,2,197,0.645902
3,Hate Crimes,0,101,0.410569
4,Hate Crimes,1,94,0.382114
5,Hate Crimes,2,51,0.207317
6,Project Safe Childhood,0,156,0.939759
7,Project Safe Childhood,1,5,0.030120
8,Project Safe Childhood,2,5,0.030120


In [39]:
pd.pivot_table(df2, values="prob", index="topics_clean", 
                   columns="topic", aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)

topic,0,1,2
topics_clean,,,
Civil Rights,0.003279,0.350820,0.645902
Hate Crimes,0.410569,0.382114,0.207317
Project Safe Childhood,0.939759,0.030120,0.030120


In [40]:
lda_display = gensimvis.prepare(ldamod, corpus_fromdict, text_raw_dict)
pyLDAvis.display(lda_display)

/Users/madhvimalhotra/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/madhvimalhotra/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/madhvimalhotra/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/madhvimalhotra/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


# 3. Extend the analysis from unigrams to bigrams (9 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [41]:
## your code here 
def create_bigram_onedoc(one_text2):
    one_text2=one_text2.split()
    bigram=""
    for first, second in zip(one_text2,one_text2[1:]):
        #bigram.append([first,second])
        bigram = bigram + first + "_" + second +" "
    return bigram

doj_subset_wscore["processed_text_bigrams"]=doj_subset_wscore["processed_text"].apply(lambda x: create_bigram_onedoc(x))


In [42]:
#doj_subset_wscore

doj_subset_wscore[doj_subset_wscore["id"] == "16-217"][["id","processed_text","processed_text_bigrams"]]


,id,processed_text,processed_text_bigrams
6727,16-217,reach comprehens settlement agreement miami miami polic resolv offic involv shoot offic announc princip deputi gener vanita gupta wifredo ferrer southern florida settlement approv miami commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu identifi pattern practic excess forc offic involv shoot violat fourth amend constitut complianc settlement monitor independ review former tampa florida polic chief castor settlement agreement implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quickli investig offic involv shoot occur measur includ settlement repres renew commit miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform princip deputi gener gupta agreement strengthen relationship commun serv improv account offic weapon unlaw provid commun particip enforc agreement today agreement result joint effort miami ensur miami polic continu effort commun protect sacr constitut citizen ferrer oversight commun agreement seek perman posit chang former chief orosa chief llane applaud commiss settlement agreement build import reform implement sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_miami miami_miami miami_polic polic_resolv resolv_offic offic_involv involv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_gener gener_vanita vanita_gupta gupta_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_offic offic_involv involv_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_offic offic_involv involv_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_castor castor_settlement settlement_agreement agreement_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_offic offic_involv involv_shoot shoot_effect effect_quickli quickli_investig investig_offic offic_involv involv_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_princip princip_deputi deputi_gener gener_gupta gupta_agreement agreement_strengthen strengthen_relationship relationship_commun commun_serv serv_improv improv_account account_offic offic_weapon weapon_unlaw unlaw_provid provid_commun commun_particip particip_enforc enforc_agreement agreement_today today_agreement agreement_result result_joint joint_effort effort_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_commun commun_protect protect_sacr sacr_constitut constitut_citizen citizen_ferrer ferrer_oversight oversight_commun commun_agreement agreement_seek seek_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_applaud applaud_commiss commiss_settlement settlement_agreement agreement_build build_import import_reform reform_implement implement_sinc sinc_issu issu_find find_includ includ_conduct conduct_attorney attorney_staff staff_special special_litig litig_section section_southern

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the 
 (1) dimensions of the `dtm` matrix from question 2.2  and 
 (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2


In [43]:
## your code here

dtm_bigram = create_dtm(list_of_strings= doj_subset_wscore.processed_text_bigrams,
                metadata = 
                doj_subset_wscore[['id', 'compound_score', 'topics_clean']])

dtm_bigram.head()


,index,id,compound_score,topics_clean,aaron_blalack,aaron_butler,aaron_conspir,aaron_decid,aaron_dylan,aaron_gaskamp,...,zone_religi,zone_request,zone_restrict,zone_student,zone_unreason,zone_varianc,zunggeemog_prompt,zunggeemog_womack,zunggeemog_write,zwengel_princeton
0,329,14-248,-0.9950,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11593,16-718,-0.9968,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,572,13-312,-0.9984,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11876,15-1348,-0.9967,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,501,11-626,-0.9985,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
print(dtm_nopre.shape)
print(dtm_bigram.shape)

(717, 6342)
(717, 65622)


# In both the cases the rows are same as expected.

There is a change in columns as bigram have more word combinations as compared to unigram.


In [45]:
for crime in ["Project Safe Childhood","Civil Rights","Hate Crimes"]:
    prevalent_bigram = get_topwords(dtm_bigram[dtm_bigram["topics_clean"]==crime],"compound_score",0.95,["index","id","topics_clean","compound_score"],"GT")    
    print("prevalent_bigrams for ", crime, " are:\n",prevalent_bigram)
    

prevalent_bigrams for  Project Safe Childhood  are:
 child_pornographi    33
project_childhood    24
child_exploit        20
gener_holder         13
child_sexual         12
sexual_exploit       12
holder_present       10
exploit_children     10
present_award        10
amber_alert          10
dtype: int64
prevalent_bigrams for  Civil Rights  are:
 settlement_agreement    28
african_american        21
deputi_gener            16
princip_deputi          16
vanita_gupta            15
gener_vanita            15
state_local             14
shelter_workshop        14
consent_decre           13
person_disabl           13
dtype: int64
prevalent_bigrams for  Hate Crimes  are:
 african_american    22
macedonia_church    16
plead_guilti        13
church_christ       11
cross_burn          10
obama_elect          7
year_prison          7
barack_obama         7
burn_macedonia       7
unit_state           6
dtype: int64


# 4. Optional extra credit 1 (1 point)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [46]:
## your code here

# 5. Optional extra credit 2 (3 points)

Review the scraping code here: https://github.com/jbencina/dojreleases/blob/master/scraper.py
    
Write code to scrape press releases from more recent years from the DOJ website than the years available in the combined.json and produce a visualization of how top words or themes in the press releases changed across the Trump administration (2016-December 2020) versus Biden administration (January 2021 onwards)- you can do this in a .py file that you submit separately and just read in the data produced by that scraping